The goal of this TD is to code and play with [PointNet](https://arxiv.org/pdf/1612.00593.pdf), a standard architecture for deep learning for pointcloud.

In this lab, we will ask you to provide both:
- notebook with logs (training logs in particular)
- the weights of the trained networks (classifier and segmentation). We provide guidance for saving the logs directly in your google drive, you can also download them via the colab interface

It should be helpful to use the slides on the side of this TD to obtain additionnal illustrations.

The idea of PointNet is to learn functions which are by definition fit to process unordered set of elements, that is functions of the shape
$$
f\big(\left\{x_1,\dots, x_n\right\}\big) = g\big(h(x_1),\dots, h(x_n)\big)
$$

where $g$ is a *symmetric* function, such as the $\max$ function.

In this TD, we will build the network as presented in the following image (slide 34):
![](https://www.lix.polytechnique.fr/Labo/Robin.Magnet/INF631/TD4/PointNet_architecture.PNG)

It consists in:
1. An input / feature transformation module
3. Per-vertex MLP
4. Feature aggregation for classification

In [1]:
!jupyter server list
import torch

Currently running servers:
http://dgxa100:8888/ :: /raid/home/bournez_pie/mva_geom/mva_geom_24/TD4


In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt

import torch.nn as nn

from torch.utils.data import DataLoader

# 1. PointNet Feature Extractor

## 1.1 Input and feature Transformation

The feature transformation module takes as input a $d$ dimensional point-cloud and regresses a $d\times d$ transformation matrix $T$.

If the input consists in $[x,y,z]$ coordinates, we can expect the network to learn a $3\times 3$ matrix which for instance rotates the point cloud in a *canonical* orientation.

If the input consists in $d$-dimensional features, the transformation matrix is harder to visualize.

![](https://www.lix.polytechnique.fr/Labo/Robin.Magnet/INF631/TD4/feature_transform.PNG)

This is done in two steps:
1. Apply per-vertex MLP to obtain a $d_{out}$ dimensional embedding for each vertex
2. use MaxPooling over the first dimension to to obtain a simple $d_{out}$ dimensional vector **for the whole point cloud**
3. Use MLP to obtain a $d^2$ dimensional vector, which defines T (flattened).
---


### Question 1

Implement the `FeatTransform` module.

**NOTE :**

Except for the **last** linear layer of the second MLP, all layers consist in of the combination of `layer`, `Batchnorm` and `ReLu` activations. The last layer is a simple linear layer with **no activation** or batchnorm (we don't want to constraint the values of matrix T).

An important (and annoying) feature in [`nn.BatchNorm1d`](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html) is that is expects the features to lie on the **second** dimension (the one after the batch). On the contrary fully connected layers expect features to lie on the **last** dimension. Unless the input is 2d, we would need to reshape values for each layer. Hopefully there is a small trick to handle this.

The trick consist in using [`nn.Conv1d`](https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html) with kernel size of $1$ to define fully connected layers. This is exactly equivalent to fully connected layers although a bit surprising at first sight. The adventage is that torch convolutions expect features to be on the **second** dimension.

Follow the following guide (or reshape at every layer, as you wish):

1. For the first MLP, the input is given with shape `(B,d,n)`. The features are in the second dimension, so use [`nn.Conv1d`](https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html), which allows to use [`nn.BatchNorm1d`](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html) directly without reshaping.
2. In the second layer you can give as input a tensor of shape `(B,d_out)` (instead of `(B,1,d_out)`). Both  [`nn.Linear`](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) and [`nn.BatchNorm1d`](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html) will work since the **second** and **last** dimensions are the same.
3. Check the documentation of [`torch.max`](https://pytorch.org/docs/stable/generated/torch.max.html), as it does not return the same thing as numpy (use `torch.max(a,dim=dim).values`).
4. We add an argument to potentially remove batchnorm (for future experiments using batch size of 1)


To stabilize training, we actually output $I + T$ so that at initialization when $T\simeq 0$ this module has no effect.


In [3]:
class MLP_con1D_Trick(nn.Module):
    # TODO
    def __init__(
        self,
        inp_dim=3,
        hidden_dims1=[64, 128, 1024],
        use_bn=True,
        activation=nn.ReLU,
        last_layer_acti=True,
    ):
        super().__init__()
        layers_list = []
        self.inp_dim = inp_dim

        if self.inp_dim != hidden_dims1[0]:
            layers_list.append(
                torch.nn.Conv1d(
                    in_channels=self.inp_dim,
                    out_channels=hidden_dims1[0],
                    kernel_size=1,
                )
            )
            layers_list.append(activation())
        for k in range(len(hidden_dims1) - 1):
            if use_bn:
                layers_list.append(nn.BatchNorm1d(num_features=hidden_dims1[k]))

            layers_list.append(
                torch.nn.Conv1d(
                    in_channels=hidden_dims1[k],
                    out_channels=hidden_dims1[k + 1],
                    kernel_size=1,
                )
            )

            if k == len(hidden_dims1) - 2:
                if last_layer_acti:
                    layers_list.append(activation())
            else:
                layers_list.append(activation())

        self.first_mlp = nn.Sequential(*layers_list)

    def forward(self, x):
        return self.first_mlp(x)

In [4]:
class FeatTransform(nn.Module):
    # TODO
    def __init__(
        self,
        inp_dim=3,
        hidden_dims1=[64, 128, 1024],
        hidden_dims2=[512, 256],
        use_bn=True,
    ):
        """
        inp_dim : int - dimension d of the input
        hidden_dims1 : list - hidden layers (inluding d_out) of the first MLP block (defined with nn.Conv1d...)
        hidden_dims2 : list - hidden layers (without d_out) of the second MLP block
        use_bn       : bool - whether to use batchnorm
        """
        super().__init__()

        ## TODO
        self.inp_dim = inp_dim
        self.use_bn = use_bn
        activation = nn.ReLU
        layers_list = []
        layers_list.append(
            torch.nn.Conv1d(
                in_channels=self.inp_dim, out_channels=hidden_dims1[0], kernel_size=1
            )
        )
        layers_list.append(activation())
        for k in range(len(hidden_dims1) - 1):
            if use_bn:
                layers_list.append(nn.BatchNorm1d(num_features=hidden_dims1[k]))

            layers_list.append(
                torch.nn.Conv1d(
                    in_channels=hidden_dims1[k],
                    out_channels=hidden_dims1[k + 1],
                    kernel_size=1,
                )
            )
            if k < len(layers_list) - 1:
                layers_list.append(activation())

        self.first_mlp = nn.Sequential(*layers_list)

        # a max pooling will happen

        layers_list = []
        layers_list.append(
            nn.Linear(in_features=hidden_dims1[-1], out_features=hidden_dims2[0])
        )
        layers_list.append(activation())
        for k in range(len(hidden_dims2) - 1):
            if use_bn:
                layers_list.append(nn.BatchNorm1d(num_features=hidden_dims2[k]))
            layers_list.append(
                nn.Linear(in_features=hidden_dims2[k], out_features=hidden_dims2[k + 1])
            )

            if k < len(layers_list) - 1:
                layers_list.append(activation())

        layers_list.append(
            nn.Linear(in_features=hidden_dims2[-1], out_features=inp_dim**2)
        )
        self.second_mlp = nn.Sequential(*layers_list)

    def forward(self, x):
        """
        x : (B, d, n) - This is standard shape for convolution inputs

        Output
        ------------
        (B, d , d) : output T defined as I + NET(x) for stability
        """
        X = self.first_mlp(x)
        # apply max pool
        X = torch.max(X, dim=-1).values
        # apply second MLP
        T = self.second_mlp(X)
        T = T.reshape((-1, self.inp_dim, self.inp_dim))
        return T + torch.eye(self.inp_dim, device=T.device).unsqueeze(0)

In [5]:
# Try a forward pass
FeatTransform()(torch.rand(2, 3, 10))  # size N,D feature, 10 POINTS

tensor([[[ 0.9348,  0.1986, -0.1241],
         [ 0.0500,  0.8737,  0.2407],
         [ 0.2258, -0.0614,  0.9890]],

        [[ 1.0244,  0.0016, -0.1256],
         [ 0.2962,  1.1618,  0.0635],
         [ 0.0383,  0.1369,  1.1079]]], grad_fn=<AddBackward0>)

## 1.2 PointNet Feature extractor

We now build the PointNet feature extractor network (output is `global_feature`)
![](https://www.lix.polytechnique.fr/Labo/Robin.Magnet/INF631/TD4/PointNet_architecture.PNG)


It can be described as follows:
1. Predict first feature transform and apply to the input
2. Apply a first 2-layer MLP with BatchNorm + ReLu.
3. Preduct second feature transform and apply to the features
4. Apply a second MLP with Batchnorm + ReLu, **except** on last layer where there is no activation (`maxpool`will serve as non-linearity).
5. Apply Maxpooling to obtain the global feature vector

Note that for future segmentation task, we will also need to outpu the second transformed features (or the input to the second MLP).

---

### Question 2
Implement the `PointNetfeat` module with the following convention

1. Use again [`nn.Conv1d`](https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html) for both MLP blocks.
2. The **first** MLP will be a 1-layer MLP with output size being the input size of the second MLP (the picture has a typo somehow)
3. The module returns the transformed features before theY are fed to the second MLP, the transformation matrix of the second block, and the global feature.
4. Both Feature Transform modules use the same hidden dimensions for their MLPs.
5. You can hardcode to default parameters, we will never change them


In [6]:
class PointNetfeat(nn.Module):
    ## TODO
    def __init__(
        self,
        inp_dim=3,
        mlp2_hidden_dims=[64, 128, 1024],
        transf_hidden_dims1=[64, 128, 1024],
        transf_hidden_dims2=[512, 256],
        use_bn=True,
    ):
        """
        inp_dim   : int - dimension d of the input
        mlp2_hidden_dims : list - hidden layers (inluding d_out) of the second MLP block (defined with nn.Conv1d...)
        transf_hidden_dims1 : list - hidden layers (inluding d_out) of the first mlp in each FeatureTransform block
        transf_hidden_dims2 : list - hidden layers (without d_out) of the second mlp in each FeatureTransform block
        use_bn : bool - whether to use batchnorm
        """
        super().__init__()
        activation = nn.ReLU
        # USEFULE INFO
        self.inp_dim = inp_dim
        self.latent_dim1 = mlp2_hidden_dims[0]  # size of the second features
        self.latent_dim = mlp2_hidden_dims[-1]  # is also the out_dim
        self.use_bn = use_bn

        # first actiation layer
        self.first_transform = FeatTransform(
            inp_dim=self.inp_dim,
            hidden_dims1=transf_hidden_dims1,
            hidden_dims2=transf_hidden_dims2,
            use_bn=use_bn,
        )  #!
        # 2. The **first** MLP will be a 1-layer MLP with output size being the input size of the second MLP (the picture has a typo somehow)
        self.first_mlp = MLP_con1D_Trick(
            inp_dim=inp_dim,
            hidden_dims1=[self.latent_dim1],
            use_bn=use_bn,
            activation=activation,
        )
        #  3. The module returns the transformed features before theY are fed to the second MLP, the transformation matrix of the second block, and the global feature.
        self.feature_transform = FeatTransform(
            inp_dim=self.latent_dim1,
            hidden_dims1=transf_hidden_dims1,
            hidden_dims2=transf_hidden_dims2,
            use_bn=use_bn,
        )
        # 4. Second mlp

        self.second_mlp = MLP_con1D_Trick(
            inp_dim=self.latent_dim1,
            hidden_dims1=mlp2_hidden_dims,
            use_bn=use_bn,
            activation=activation,
            last_layer_acti=False,
        )

        # 5. You can hardcode to default parameters, we will never change them

    def forward(self, x):
        """
        Parameters
        --------------
        x : (B, in_dim, n) input batch in convolution format.

        Output
        -------------
        global_feature : (B, d_out) output global feature (d_out=1024)
        T_feat : (B, latent1, latent1) transformation matrix of the second FeatureTransform module
        x_feat : (B, latent1, n) per-point features after being transformed by the second
                  FeatureTransform module
        """

        T = self.first_transform(x)  # transflation

        x = torch.einsum("ijk, ijl -> ijk", x, T)  ## TODO
        # first mlp with 1 layer
        x = self.first_mlp(x)
        # predict second feature transform
        T_feat = self.feature_transform(x)
        x_feat = torch.einsum("ijk, ijl -> ijk", x, T_feat)  ## TODO
        # apply a second MLP without maxpool at last_layers
        x = self.second_mlp(x)
        # apply max_pooling as global feature vecotr

        global_feature = torch.max(x, dim=-1).values
        return global_feature, T_feat, x_feat


global_, T_, x_ = PointNetfeat()(torch.rand(2, 3, 2048))

In [7]:
# Try a forward pass
global_, T_, x_ = PointNetfeat()(torch.rand(2, 3, 2048))
print(global_.shape, T_.shape, x_.shape)  # check the shapes

torch.Size([2, 1024]) torch.Size([2, 64, 64]) torch.Size([2, 64, 2048])


# 2 PointNet Classifier

## 2.1 Network Design

The PointNet Classifier is the complete network seen as follow
![](https://www.lix.polytechnique.fr/Labo/Robin.Magnet/INF631/TD4/PointNet_architecture.PNG)


### Question 3
Implement the `PointNetCls` module.
1. The last MLP also uses BatchNorm + ReLu, **except** for the last layer, which uses either `log_softmax` directly or `softmax`, or even nothing. Note that the training loss you'll chose will be adapted to this choice !
2. You can hardcode to default parameters, we will not change them
3. We don't need the transformed feature, only the transformation matrix for classification

In [8]:
class PointNetCls(nn.Module):
    ## TODO
    def __init__(
        self,
        n_cls,
        inp_dim=3,
        cls_hidden_dims=[512, 256],
        mlp2_hidden_dims=[64, 128, 1024],
        transf_hidden_dims1=[64, 128, 1024],
        transf_hidden_dims2=[512, 256],
        use_bn=True,
    ):
        """
        PointNet classification network

        Parameters
        ------------------
            n_cls (int): Number of classes for classification
            inp_dim (int): Dimension of the input points
            cls_hidden_dims (list): List of hidden dimensions for the MLP classifier
            mlp2_hidden_dims (list): List of hidden dimensions for the second MLP block in the feature extractor
            transf_hidden_dims1 (list): List of hidden dimensions for the first MLP in each FeatureTransform block
            transf_hidden_dims2 (list): List of hidden dimensions for the second MLP in each FeatureTransform block
            use_bn (bool): Whether to use batchnorm
        """

        super(PointNetCls, self).__init__()
        self.use_bn = use_bn
        activation = nn.ReLU
        self.net = PointNetfeat(
            inp_dim=inp_dim,
            mlp2_hidden_dims=mlp2_hidden_dims,
            transf_hidden_dims1=transf_hidden_dims1,
            transf_hidden_dims2=transf_hidden_dims2,
            use_bn=use_bn,
        )

        output_dim = mlp2_hidden_dims[-1]
        layers_list = []
        layers_list.append(
            nn.Linear(in_features=output_dim, out_features=cls_hidden_dims[0])
        )
        layers_list.append(activation())
        for k in range(len(cls_hidden_dims) - 1):
            if use_bn:
                layers_list.append(nn.BatchNorm1d(num_features=cls_hidden_dims[k]))
            layers_list.append(
                nn.Linear(
                    in_features=cls_hidden_dims[k], out_features=cls_hidden_dims[k + 1]
                )
            )
            if k < len(cls_hidden_dims): 
                layers_list.append(activation())
        # ADD A SOFTMAX IF NEEDS BE
        # we have to project  to n_cls
        layers_list.append(
            nn.Linear(in_features=cls_hidden_dims[-1], out_features=n_cls)
        )
        self.last_mlp = nn.Sequential(*layers_list)

    def forward(self, x):
        """
        Parameters
        --------------
        inp : (B, n, d) input batch in standard format.

        Output
        -------------
        out : (B, n_cls) output score for each class
        T_feat : (B, latent1, latent1) transformation matrix of the second FeatureTransform module
        """
        x = x.transpose(2, 1)  # Put input in convolution format
        x, T_feat, x_feat = self.net(x)
        out = self.last_mlp(x)
        return out, T_feat


global_, T_ = PointNetCls(n_cls=5)(torch.rand(2, 64, 3))

## 2.2 - Data Augmentation

Since PointNet takes $(X,Y,Z)$ coordinates as input, it is not invariant to rigid transformations of the point cloud. One possibility is to make it learn to be invariant, by rigidly modifying shapes during training.

### Question 4
Fill the random scaling and translation function using numpy.
1. Scale and translate each coordinate independantly
2. Scaling factor lies between $\frac{2}{3}$ and $\frac{3}{2}$
2. Translation lies between $-0.2$ and $0.2$

In [9]:
def random_scale_transl(X):
    """Apply random scaling to the point cloud.
    Scaling is applied for

    Args:
        X (np.ndarray): Point cloud data, (N, 3).

    Returns:
        X_transformed (np.ndarray): Scaled (and translated) point cloud data (N, 3).
    """
    scale = np.random.uniform(low=2 / 3, high=3 / 2)
    translation = np.random.uniform(low=-0.2, high=0.2)
    X_transformed = X * scale + translation
    return X_transformed


X = np.eye(N=3)
random_scale_transl(X)

array([[ 0.64511156, -0.08606346, -0.08606346],
       [-0.08606346,  0.64511156, -0.08606346],
       [-0.08606346, -0.08606346,  0.64511156]])

### Question 5
For this specfific dataset, all shapes pre-aligned, and only variability is rotation along the $y$ axis.

Therefore simply generate a random rotation matrix along the y axis.

In [10]:
def random_rotation(X):
    # TODO
    """Apply random rotation to the point cloud (along the y axis)

    Args:
        X (np.ndarray): Point cloud data, (N, 3).

    Returns:
        X_rot (np.ndarray): Rotated point cloud data (N, 3).
    """
    rotation = np.random.uniform(low=0, high=2 * np.pi)
    rot = np.array(
        [
            [np.cos(rotation), 0, np.sin(rotation)],
            [0, 1, 0],
            [-np.sin(rotation), 0, np.cos(rotation)],
        ]
    )
    X_rot = rot.dot(X.T).T
    return X_rot

## 2.3 Data Loading

We will train on the [ModelNet40](https://www.cv-foundation.org/openaccess/content_cvpr_2015/papers/Wu_3D_ShapeNets_A_2015_CVPR_paper.pdf) dataset.

This dataset consist in pointclouds of 12000 objects of 40 different categories such as airplane, car, plant, lamp. Because we only need to provide a global feature vector for each shape, we can subsample each surface with a fixed number of parameters, which allows us to use large batch sizes.

![](https://production-media.paperswithcode.com/datasets/modelnet.jpeg)


We provide below code for dataloading.

Let's first download the data and define the loaders.

In [11]:
!pip install h5py

In [12]:
!mkdir 'data'
!wget -O './data/modelnet40_ply_hdf5_2048.zip' https://huggingface.co/datasets/Msun/modelnet40/resolve/main/modelnet40_ply_hdf5_2048.zip?download=true --no-check-certificate
!unzip -n data/modelnet40_ply_hdf5_2048.zip -d ./data/ 
#! # UNCOMMENT IT IF YOU WANT

mkdir: cannot create directory ‘data’: File exists
--2024-11-04 14:07:21--  https://huggingface.co/datasets/Msun/modelnet40/resolve/main/modelnet40_ply_hdf5_2048.zip?download=true
Resolving huggingface.co (huggingface.co)... 18.155.129.31, 18.155.129.4, 18.155.129.60, ...
Connecting to huggingface.co (huggingface.co)|18.155.129.31|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/36/90/3690071fa5fce49ab533e0e81bd5b7fd1fc7337b68386704b06e7c4dfe6eed96/f01b8189281fae5790e39deb9f3eca86e446b771bdc665c6ad05f28d039b20e7?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27modelnet40_ply_hdf5_2048.zip%3B+filename%3D%22modelnet40_ply_hdf5_2048.zip%22%3B&response-content-type=application%2Fzip&Expires=1730984841&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMDk4NDg0MX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy8zNi85MC8zNjkwMDcxZmE1ZmNlNDlhYjUzM2UwZTgxYmQ1YjdmZDFmYzczMzd

In [13]:
import h5py
import os.path as osp

from pathlib import Path
from tqdm.auto import tqdm
from torch.utils.data import Dataset


def load_h5(h5_filename):
    f = h5py.File(h5_filename)
    print(h5_filename)
    data = f["data"][:]
    label = f["label"][:]
    f.close()
    return (data, label)


def load_h5_files(data_path, files_list_path):
    """Load h5 into memory"""
    files_list = [
        Path(line.rstrip()).name for line in open(osp.join(data_path, files_list_path))
    ]
    data = []
    labels = []
    for i in range(len(files_list)):
        data_, labels_ = load_h5(os.path.join(data_path, files_list[i]))
        data.append(data_)
        labels.append(labels_)
    data = np.concatenate(data, axis=0)
    labels = np.concatenate(labels, axis=0)
    return data, labels


class ModelNet40Generator(Dataset):
    def __init__(
        self, mode, data_dir, files_list, num_classes=40, num_points=1024, rot_aug=False
    ):
        """Initialize the ModelNet40Generator class.

        Args:
            mode (str): 'train' or 'test'.
            data_dir (str): Path to the data directory.
            files_list (str): Path to the files list.
            num_classes (int, optional): ModelNet class. Defaults to 40.
            num_points (int, optional): Input PC. Defaults to 1024.
            rot_aug (bool, optional): Augment data. Defaults to False.
        """

        assert mode.lower() in ("train", "val", "test")
        assert files_list in ("train_files.txt", "test_files.txt")
        self.data, labels = load_h5_files(data_dir, files_list)

        self.mode = mode
        self.num_points = num_points
        self.num_classes = num_classes
        self.num_samples = self.data.shape[0]
        self.rot_aug = rot_aug
        self.labels = np.reshape(labels, (-1,))

    def __len__(self) -> int:
        return self.num_samples

    def __getitem__(self, idx):
        indexes = np.random.permutation(np.arange(self.num_points))[: self.num_points]
        X = self.data[idx, indexes, ...]
        y = self.labels[idx, ...]
        y_categorical = torch.from_numpy(
            np.eye(self.num_classes, dtype="uint8")[y]
        ).long()

        if self.mode == "train" and self.rot_aug:
            # X = self.random_scaling(X)
            X = self.random_rotation_3d(X)

        return torch.from_numpy(X).float(), y_categorical.float()

    def random_scaling(self, X):
        """Apply random scaling to the point cloud.

        Args:
            X (np.ndarray): Point cloud data, (N, 3).

        Returns:
            X (np.ndarray): Scaled point cloud data (N, 3).
        """
        return random_scale_transl(X)

    def random_rotation_3d(self, X):
        """Apply random rotation to the point cloud (axis agnostic).

        Args:
            X (np.ndarray): Point cloud data, (N, 3).

        Returns:
            rotated_data (np.ndarray): Rotated point cloud data (N, 3).
        """
        return random_rotation(X)

## 2.4. Classification Loss

PointNet classification loss is defined as the weighted sum of two losses: the classification loss $L_{cls}$, such as cross-entropy, and a regularization loss $L_{reg}$ which regularizes the FeatureTransform output.

In particular, the the regularization loss forces the (second) predicted transformation matrix $T$ to be near-orthogonal (so close to a "rotation"):

$$
L_{reg}(T) = \|T^\top T - I\|_2^2
$$

This loss is averaged along the batch.

This regularization is only applied to the second transformation matrix, that is the one applied to the feature space.

### Question 6
Code the classification loss. Adapt $L_{cls}$ to whatever activation you used in you architecture.

In [14]:
def classification_loss(preds, gt_labels, T_feat, w_reg):
    """
    Classification loss for PointNet

    Parameters
    --------------
    preds : (B, n_cls) - output of the classifier
    gt_labels : (B,) - ground truth labels
    T_feat : (B, latent1, latent1) - transformation matrix of the second FeatureTransform module
    w_reg : weight of the regularization term

    Output
    -------------
    loss : classification loss L_{cls} + w_reg * L_{reg}
    """
    loss1 = nn.CrossEntropyLoss()

    x = torch.eye(T_feat.shape[1], device=T_feat.device)
    x = x.reshape((1, T_feat.shape[1], T_feat.shape[1]))
    id = x.repeat(T_feat.shape[0], 1, 1)
    T = torch.einsum("ijk, ikj -> ijk", T_feat, T_feat.transpose(1, 2))
    norm = torch.linalg.norm(T - id)  # I don't think we need to specify a dim
    return loss1(preds, gt_labels) + w_reg * norm


B, n_cls, latent1 = 4, 10, 3  # Batch size, number of classes, latent dimension
preds = torch.rand(B, n_cls)  # Random predictions
gt_labels = torch.randint(0, n_cls, (B,))  # Random ground truth labels
T_feat = torch.rand(B, latent1, latent1)  # Random transformation matrix
w_reg = 0.1  # Regularization weight

# Call the function
loss = classification_loss(preds, gt_labels, T_feat, w_reg)
loss

tensor(2.9842)

## 2.5 Training

This cell defines the trainer class, using your classification loss, simply run it.

In [15]:
class Trainer(object):
    def __init__(
        self,
        train_loader,
        valid_loader,
        device="cuda",
        lr=1e-3,
        weight_decay=1e-4,
        num_epochs=200,
        lr_decay_every=50,
        lr_decay_rate=0.5,
        log_interval=10,
        save_dir=None,
        **model_cfg,
    ):
        """
        train_loader: (torch.utils.DataLoader) DataLoader for training set
        valid_loader: (torch.utils.DataLoader) DataLoader for validation set
        device: (str) 'cuda' or 'cpu'
        lr: (float) learning rate
        weight_decay: (float) weight decay for optimiser
        num_epochs: (int) number of epochs
        lr_decay_every: (int) decay learning rate every this many epochs
        lr_decay_rate: (float) decay learning rate by this factor
        log_interval: (int) print training stats every this many iterations
        save_dir: (str) directory to save model checkpoints
        model_cfg: (dict) keyword arguments for model
        """
        self.model = self.get_model(model_cfg).to(device)
        self.train_loader = train_loader
        self.valid_loader = valid_loader
        self.device = device
        self.lr = lr
        self.weight_decay = weight_decay
        self.num_epochs = num_epochs

        self.optimizer = torch.optim.Adam(
            self.model.parameters(), lr=self.lr, weight_decay=self.weight_decay
        )

        self.lr_decay_every = lr_decay_every
        self.lr_decay_rate = lr_decay_rate
        self.log_interval = log_interval
        self.save_dir = save_dir
        if not os.path.exists(self.save_dir):
            os.makedirs(self.save_dir)

        self.train_losses = []
        self.val_losses = []
        self.train_accs = []
        self.val_accs = []

        self.inp_feat = model_cfg.get("inp_feat", "xyz")
        self.num_eig = model_cfg.get("num_eig", 128)
        if self.inp_feat not in ["xyz", "xyzn", "hks", "wks"]:
            raise ValueError("inp_feat must be one of xyz, xyzn, hks, wks")

        self.model.to(self.device)

    def get_model(self, model_cfg):
        if model_cfg["name"].lower() == "pointnet":
            return PointNetCls(
                n_cls=model_cfg["n_cls"],
                inp_dim=model_cfg["inp_dim"],
                cls_hidden_dims=model_cfg["cls_hidden_dims"],
                mlp2_hidden_dims=model_cfg["mlp2_hidden_dims"],
                transf_hidden_dims1=model_cfg["transf_hidden_dims1"],
                transf_hidden_dims2=model_cfg["transf_hidden_dims2"],
            )
        else:
            raise ValueError(
                "%s must be one of PointNet, PointNet++" % model_cfg["name"]
            )

    def adjust_lr(self):
        lr = self.lr * self.lr_decay_rate
        for param_group in self.optimizer.param_groups:
            param_group["lr"] = lr

    def forward_step(self, inp):
        """
        Perform a forward step of the model.

        Args:
            inp (torch.Tensor): (N, D) tensor of input features

        Returns:
            pred (torch.Tensor): (N, p_out) tensor of predicted labels.
        """
        inp = inp.to(self.device)
        preds, trans_feat = self.model(inp)
        return preds, trans_feat

    def get_loss(self, preds, gt_labels, trans_feat):
        loss = classification_loss(preds, gt_labels.argmax(dim=1), trans_feat, 1e-3)
        return loss

    def train_epoch(self):
        train_loss = 0
        train_acc = 0
        for i, (inp_pts, gt_labels) in enumerate(tqdm(self.train_loader)):
            self.optimizer.zero_grad()
            gt_labels = gt_labels.to(self.device)
            preds, trans_feat = self.forward_step(inp_pts)
            loss = self.get_loss(preds, gt_labels, trans_feat)
            loss.backward()
            self.optimizer.step()
            train_loss += loss.item()
            pred_labels = torch.max(preds, dim=1).indices
            this_correct = pred_labels.eq(gt_labels.argmax(dim=-1)).sum().item()
            train_acc += this_correct / gt_labels.shape[0]

        return train_loss / len(self.train_loader), train_acc / len(self.train_loader)

    def valid_epoch(self):
        val_loss = 0
        val_acc = 0
        for i, (inp_pts, gt_labels) in enumerate(tqdm(self.valid_loader, leave=False)):
            gt_labels = gt_labels.to(self.device)
            preds, trans_feat = self.forward_step(inp_pts)
            loss = self.get_loss(preds, gt_labels, trans_feat)
            val_loss += loss.item()
            pred_labels = torch.max(preds, dim=1).indices
            this_correct = pred_labels.eq(gt_labels.argmax(dim=-1)).sum().item()
            val_acc += this_correct / gt_labels.shape[0]

        return val_loss / len(self.valid_loader), val_acc / len(self.valid_loader)

    def run(self):
        for epoch in tqdm(range(self.num_epochs)):
            self.model.train()

            if epoch % self.lr_decay_every == 0:
                self.adjust_lr()

            train_ep_loss, train_ep_acc = self.train_epoch()
            self.train_losses.append(train_ep_loss)
            self.train_accs.append(train_ep_acc)

            if epoch % self.log_interval == 0:
                val_loss, val_acc = self.valid_epoch()
                self.val_losses.append(val_loss)
                self.val_accs.append(val_acc)
                torch.save(
                    self.model.state_dict(),
                    os.path.join(self.save_dir, "model_latest.pth"),
                )
                print(
                    "Epoch: {:03d}, Train Loss: {:.4f}, Train Acc: {:.2f}%, Val Loss: {:.4f}, Val Acc: {:.2f}%".format(
                        epoch,
                        train_ep_loss,
                        1e2 * train_ep_acc,
                        val_loss,
                        1e2 * val_acc,
                    )
                )
        torch.save(
            self.model.state_dict(), os.path.join(self.save_dir, "model_final.pth")
        )

    def test(self):
        file_final = os.path.join(self.save_dir, "model_final.pth")
        if not os.path.exists(file_final):
            print("-------------------------------------------------------")
            print("No final weights, switching to last weights")
            print("-------------------------------------------------------")
            file_final = os.path.join(self.save_dir, "model_latest.pth")
        weights = torch.load(file_final)
        self.model.load_state_dict(weights)
        _, score = self.valid_epoch()
        print("Final Valid Accuracy : {:.2f}%".format(1e2 * score))

Run the following cells to train the network.

In [16]:
N_CLS = 40
data_dir = "data/modelnet40_ply_hdf5_2048"
train_data = ModelNet40Generator(
    mode="train", data_dir=data_dir, files_list="train_files.txt", rot_aug=True
)
val_data = ModelNet40Generator(
    mode="val", data_dir=data_dir, files_list="test_files.txt"
)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False, num_workers=0)

data/modelnet40_ply_hdf5_2048/ply_data_train0.h5
data/modelnet40_ply_hdf5_2048/ply_data_train1.h5
data/modelnet40_ply_hdf5_2048/ply_data_train2.h5
data/modelnet40_ply_hdf5_2048/ply_data_train3.h5
data/modelnet40_ply_hdf5_2048/ply_data_train4.h5
data/modelnet40_ply_hdf5_2048/ply_data_test0.h5
data/modelnet40_ply_hdf5_2048/ply_data_test1.h5


You will need to modify this cell if you are not using Google Colab + Drive

In [17]:
# Comment these if you are going local
# from google.colab import drive
# drive.mount('/content/drive/')
## Change this if your are going local, otherwise keep it (checkpoints will be in checkpoint folder of your drive)
# save_dir = '/content/drive/MyDrive/checkpoints'
save_dir = "checkpoints"
os.makedirs(save_dir, exist_ok=True)

In [ ]:
model_cfg = dict(
    name="pointnet",
    n_cls=N_CLS,
    inp_dim=3,
    cls_hidden_dims=[512, 256],
    mlp2_hidden_dims=[64, 128, 1024],
    transf_hidden_dims1=[64, 128, 1024],
    transf_hidden_dims2=[512, 256],
)

trainer = Trainer(train_loader, val_loader, lr=0.001, weight_decay=0.0, num_epochs=100, save_dir=os.path.join(save_dir, 'classifier'), **model_cfg)
trainer.run()

KeyError: 'inp_dim'

In [ ]:
trainer.test()

# 3. PointNet Segmentation

## 3.1 Network Design

We now build the PointNet feature extractor network (slide 34, output is `global_feature`)
![](https://www.lix.polytechnique.fr/Labo/Robin.Magnet/INF631/TD4/pointnet_segmentation.PNG)


### Question 7
Build the segmentation network in the case of a batch size of 1.
**Disable all batchnorms**

In [ ]:
class PointNetSeg(nn.Module):
    ## TODO
    def __init__(
        self,
        n_cls,
        inp_dim=3,
        seg_hidden_dims=[512, 256, 128],
        mlp2_hidden_dims=[64, 128, 1024],
        transf_hidden_dims1=[64, 128, 1024],
        transf_hidden_dims2=[512, 256],
    ):
        """
        PointNet classification network

        Parameters
        ------------------
            n_cls (int): Number of classes for classification
            inp_dim (int): Dimension of the input points
            seg_hidden_dims (list): List of hidden dimensions for the segmentation MLP
            mlp2_hidden_dims (list): List of hidden dimensions for the second MLP block in the feature extractor
            transf_hidden_dims1 (list): List of hidden dimensions for the first MLP in each FeatureTransform block
            transf_hidden_dims2 (list): List of hidden dimensions for the second MLP in each FeatureTransform block
        """

        super().__init__()
        use_bn = False  #!
        activation = torch.nn.ReLU
        self.net = PointNetfeat(
            inp_dim=inp_dim,
            mlp2_hidden_dims=mlp2_hidden_dims,
            transf_hidden_dims1=transf_hidden_dims1,
            transf_hidden_dims2=transf_hidden_dims2,
            use_bn=use_bn,
        )

        layers_list = []
        output_dim = mlp2_hidden_dims[0] + mlp2_hidden_dims[-1]  # Unsure about results
        # build the segmentation layers
        layers_list.append(
            nn.Linear(in_features=output_dim, out_features=seg_hidden_dims[0])
        )
        layers_list.append(activation())
        for k in range(len(seg_hidden_dims) - 1):
            layers_list.append(
                nn.Linear(
                    in_features=seg_hidden_dims[k], out_features=seg_hidden_dims[k + 1]
                )
            )

            if k < len(seg_hidden_dims):  #! IMPORTANT -1
                layers_list.append(activation())
        # we have to project  to n_cls
        layers_list.append(
            nn.Linear(in_features=seg_hidden_dims[-1], out_features=n_cls)
        )
        self.last_mlp = nn.Sequential(*layers_list)

    def forward(self, x):
        """
        Parameters
        --------------
        inp : (B, n, d) input batch in standard format.

        Output
        -------------
        out : (B, n, n_cls) output score for each class
        T_feat : (B, latent1, latent1) transformation matrix of the second FeatureTransform module
        """
        B, n, d = x.size()

        x = x.transpose(2, 1)  # Put input in convolution format
        global_feature, T_feat, x_feat = self.net(x)
        global_feature_broadcasted = global_feature.unsqueeze(2).repeat(1, 1, n)
        x = torch.cat((global_feature_broadcasted, x_feat), dim=1).transpose(1, 2)
        out = self.last_mlp(x)
        out = out

        return out, T_feat


out_seg, T_feat = PointNetSeg(260)(torch.rand(2, 2046, 3))

## 3.2 Data Loading

We will train on the RNA dataset from Lab3. Since all shapes have different number of vertices, we can't use batch size bigger than one.

Run the following cells to download and define code to load the data.

In [ ]:
!pip install potpourri3d
# Uncomment for colab
# !pip install git+https://github.com/skoch9/meshplot.git
# !pip install pythreejs

!wget https://www.lix.polytechnique.fr/Labo/Robin.Magnet/INF631/TD3/diffusion_utils.py
!wget https://www.lix.polytechnique.fr/Labo/Robin.Magnet/INF631/TD3/RNADataset.zip
!unzip -o RNADataset.zip
!mkdir models
!wget https://www.lix.polytechnique.fr/Labo/Robin.Magnet/INF631/material_TD3.zip
!unzip -o material_TD3.zip

In [ ]:
from diffusion_utils import normalize_positions, read_mesh
from mesh_utils.mesh import TriMesh
import plot_utils as plu  # Comment if you are not visualizing the results


class RNAMeshDataset(Dataset):
    """RNA Mesh Dataset"""

    def __init__(self, root_dir, train, n_classes=260):
        """
        root_dir (string): Directory with all the meshes.
        train (bool): If True, use the training set, else use the test set.
        num_eig (int): Number of eigenvalues to use.
        op_cache_dir (string): Directory to cache the operators.
        n_classes (int): Number of classes.
        """

        self.train = train  # bool
        self.root_dir = root_dir
        self.n_class = n_classes  # (includes -1)

        # store in memory
        self.verts_list = []
        self.faces_list = []
        self.labels_list = []  # per-vertex

        # Load the meshes & labels
        if self.train:
            with open(os.path.join(self.root_dir, "train.txt")) as f:
                this_files = [line.rstrip() for line in f]
        else:
            with open(os.path.join(self.root_dir, "test.txt")) as f:
                this_files = [line.rstrip() for line in f]

        print("loading {} files: {}".format(len(this_files), this_files))

        # Load the actual files

        off_path = os.path.join(root_dir, "off")
        label_path = os.path.join(root_dir, "labels")
        for f in this_files:
            off_file = os.path.join(off_path, f)
            label_file = os.path.join(label_path, f[:-4] + ".txt")

            verts, faces = read_mesh(off_file)
            labels = np.loadtxt(label_file).astype(int) + 1  # shift -1 --> 0

            verts = torch.tensor(verts).float()
            faces = torch.tensor(faces)
            labels = torch.tensor(labels)

            # center and unit scale
            verts = normalize_positions(verts)

            self.verts_list.append(verts)
            self.faces_list.append(faces)
            self.labels_list.append(labels)

    def __len__(self):
        return len(self.verts_list)

    def __getitem__(self, idx):
        return self.verts_list[idx], self.faces_list[idx], self.labels_list[idx]

## 3.3 Segmentation Loss

PointNet segmentation loss is defined the same as the classification loss, except we know perform per-vertex classification using $L_{seg}$ instead of $L_{cls}$. Adapt again the loss to your network design


### Question 8
Code the segmentation loss. Adapt $L_{seg}$ to whatever activation you used in you architecture. You might be able to use exactly the same code.

In [ ]:
def segmentation_loss(preds, gt_labels, T_feat, w_reg):
    """
    Classification loss for PointNet

    Parameters
    --------------
    preds : (B, n_cls) - output of the segmentation network
    gt_labels : (B, n) - ground truth labels
    T_feat : (B, latent1, latent1) - transformation matrix of the second FeatureTransform module
    w_reg : weight of the regularization term

    Output
    -------------
    loss : segmentation loss L_{seg} + w_reg * L_{reg}
    """
    loss1 = nn.CrossEntropyLoss()
    # x = torch.eye(T_feat.shape[1],device=T_feat.device)
    # x = x.reshape((1, T_feat.shape[1], T_feat.shape[1]))
    # id = x.repeat(T_feat.shape[0], 1, 1)
    # T=torch.einsum("ijk, ikj -> ijk", T_feat, T_feat.transpose(1,2))

    # norm=torch.linalg.norm(T-id) #I don't think we need to specify a dim
    # #TODO: check that the activation function indeed is the right one
    B, d, _ = T_feat.shape
    identity = torch.eye(d, device=T_feat.device).unsqueeze(0)  # (1, d, d)
    orthogonal_diff = torch.bmm(T_feat, T_feat.transpose(1, 2)) - identity
    loss_reg = torch.norm(orthogonal_diff) ** 2 / B
    return loss1(preds, gt_labels) + w_reg * loss_reg


B, n_cls, n = 4, 10, 100  # Batch size, number of classes, number of points
preds = torch.randn(B, n_cls, n, requires_grad=True)  # Random predictions
gt_labels = torch.randint(0, n_cls, (B, n))
T_feat = torch.eye(64).expand(B, -1, -1)  # Identity matrix as T_feat
w_reg = 0.1  # Regularization weight
# Calculate loss
loss = segmentation_loss(preds, gt_labels, T_feat, w_reg)
loss.shape

## 3.4 Training

This defines a trainer class using your loss. Simply run the cells.

In [ ]:
class TrainerSeg(object):
    def __init__(
        self,
        train_loader,
        valid_loader,
        device="cuda",
        lr=1e-3,
        weight_decay=1e-4,
        num_epochs=200,
        lr_decay_every=50,
        lr_decay_rate=0.5,
        log_interval=10,
        save_dir=None,
        **model_cfg,
    ):
        """
        pointNet_cls: (nn.Module) class of the PointNet (or ++) model.
        train_loader: (torch.utils.DataLoader) DataLoader for training set
        valid_loader: (torch.utils.DataLoader) DataLoader for validation set
        device: (str) 'cuda' or 'cpu'
        lr: (float) learning rate
        weight_decay: (float) weight decay for optimiser
        num_epochs: (int) number of epochs
        lr_decay_every: (int) decay learning rate every this many epochs
        lr_decay_rate: (float) decay learning rate by this factor
        log_interval: (int) print training stats every this many iterations
        save_dir: (str) directory to save model checkpoints
        model_cfg: (dict) keyword arguments for model
        """
        self.model = self.get_model(model_cfg).to(device)
        self.train_loader = train_loader
        self.valid_loader = valid_loader
        self.device = device
        self.lr = lr
        self.weight_decay = weight_decay
        self.num_epochs = num_epochs

        self.optimizer = torch.optim.Adam(
            self.model.parameters(), lr=self.lr, weight_decay=self.weight_decay
        )
        self.loss = torch.nn.CrossEntropyLoss()

        self.lr_decay_every = lr_decay_every
        self.lr_decay_rate = lr_decay_rate
        self.log_interval = log_interval
        self.save_dir = save_dir
        if not os.path.exists(self.save_dir):
            os.makedirs(self.save_dir)

        self.train_losses = []
        self.val_losses = []
        self.train_accs = []
        self.val_accs = []

        self.inp_feat = model_cfg.get("inp_feat", "xyz")
        self.num_eig = model_cfg.get("num_eig", 128)
        if self.inp_feat not in ["xyz", "xyzn", "hks", "wks"]:
            raise ValueError("inp_feat must be one of xyz, xyzn, hks, wks")

        self.model.to(self.device)

    def get_model(self, model_cfg):
        if model_cfg["name"].lower() == "pointnet":
            return PointNetSeg(
                n_cls=model_cfg["n_cls"],
                inp_dim=model_cfg["inp_dim"],
                seg_hidden_dims=model_cfg["seg_hidden_dims"],
                mlp2_hidden_dims=model_cfg["mlp2_hidden_dims"],
                transf_hidden_dims1=model_cfg["transf_hidden_dims1"],
                transf_hidden_dims2=model_cfg["transf_hidden_dims2"],
            )
        else:
            raise ValueError(
                "%s must be one of PointNet, PointNet++" % model_cfg["name"]
            )

    def adjust_lr(self):
        lr = self.lr * self.lr_decay_rate
        for param_group in self.optimizer.param_groups:
            param_group["lr"] = lr

    def forward_step(self, inp):
        """
        Perform a forward step of the model.

        Args:
            inp (torch.Tensor): (N, D) tensor of input features

        Returns:
            pred (torch.Tensor): (N, p_out) tensor of predicted labels.
        """
        inp = inp.to(self.device)
        verts = inp[..., :3].to(self.device)
        norms = inp[..., 3:].to(self.device)
        preds, trans_feat = self.model(inp)
        return preds, trans_feat

    def get_loss(self, preds, gt_labels, trans_feat):
        loss = segmentation_loss(preds, gt_labels, trans_feat, 1e-3)
        return loss

    def train_epoch(self):
        train_loss = 0
        train_acc = 0
        for i, (inp_pts, _, gt_labels) in enumerate(tqdm(self.train_loader)):
            self.optimizer.zero_grad()
            gt_labels = gt_labels.to(self.device)
            preds, trans_feat = self.forward_step(inp_pts)
            loss = self.get_loss(preds[0], gt_labels[0], trans_feat)
            loss.backward()
            self.optimizer.step()
            train_loss += loss.item()

            pred_labels = torch.max(preds[0], dim=1).indices  # (N,)
            this_correct = pred_labels.eq(gt_labels).sum().item()
            train_acc += this_correct / inp_pts.shape[-2]

        return train_loss / len(self.train_loader), train_acc / len(self.train_loader)

    def valid_epoch(self):
        val_loss = 0
        val_acc = 0
        total = 0
        for i, (inp_pts, _, gt_labels) in enumerate(self.valid_loader):
            gt_labels = gt_labels.to(self.device)
            preds, trans_feat = self.forward_step(inp_pts)
            loss = self.get_loss(preds[0], gt_labels[0], trans_feat)
            val_loss += loss.item()

            pred_labels = torch.max(preds[0], dim=1).indices
            this_correct = pred_labels.eq(gt_labels).sum().item()
            total += inp_pts.shape[-2]
            val_acc += this_correct

        return val_loss / total, val_acc / total

    def run(self):
        for epoch in tqdm(range(self.num_epochs)):
            self.model.train()

            if epoch % self.lr_decay_every == 0:
                self.adjust_lr()

            train_ep_loss, train_ep_acc = self.train_epoch()
            self.train_losses.append(train_ep_loss)
            self.train_accs.append(train_ep_acc)

            if epoch % self.log_interval == 0:
                val_loss, val_acc = self.valid_epoch()
                self.val_losses.append(val_loss)
                self.val_accs.append(val_acc)
                torch.save(
                    self.model.state_dict(),
                    os.path.join(self.save_dir, "model_latest.pth"),
                )
                print(
                    "Epoch: {:03d}, Train Loss: {:.4f}, Train Acc: {:.2f}, Val Loss: {:.4f}, Val Acc: {:.2f}%".format(
                        epoch,
                        train_ep_loss,
                        1e2 * train_ep_acc,
                        val_loss,
                        1e2 * val_acc,
                    )
                )
        torch.save(
            self.model.state_dict(), os.path.join(self.save_dir, "model_final.pth")
        )

    def visualize(self):
        """
        We only test the first two shapes of validation set.
        """
        self.model.eval()
        test_seg_meshes = []

        for i, (inp_pts, inp_faces, gt_labels) in enumerate(self.valid_loader):
            gt_labels = gt_labels.to(self.device)
            preds, trans_feat = self.forward_step(inp_pts)
            pred_labels = torch.max(preds[0], dim=1).indices
            test_seg_meshes.append(
                [
                    TriMesh(
                        inp_pts.squeeze().cpu().numpy(),
                        inp_faces.squeeze().cpu().numpy(),
                    ),
                    pred_labels.squeeze().cpu().numpy(),
                ]
            )
            if i == 1:
                break

        cmap1 = plt.get_cmap("jet")(test_seg_meshes[0][-1] / (146))[:, :3]
        cmap2 = plt.get_cmap("jet")(test_seg_meshes[1][-1] / (146))[:, :3]

        plu.double_plot(test_seg_meshes[0][0], test_seg_meshes[1][0], cmap1, cmap2)

    def test(self):
        file_final = os.path.join(self.save_dir, "model_final.pth")
        if not os.path.exists(file_final):
            print("-------------------------------------------------------")
            print("No final weights, switching to last weights")
            print("-------------------------------------------------------")
            file_final = os.path.join(self.save_dir, "model_latest.pth")
        weights = torch.load(file_final)
        self.model.load_state_dict(weights)
        _, score = self.valid_epoch()
        print("Final Valid Segmentation Accuracy : {:.2f}%".format(1e2 * score))

Run these cells to see

In [ ]:
root_dir = "./RNADataset/"

train_dataset = RNAMeshDataset(root_dir, train=True)
train_loader = DataLoader(
    train_dataset, batch_size=1, shuffle=True, num_workers=0, persistent_workers=False
)

valid_dataset = RNAMeshDataset(root_dir, train=False)
valid_loader = DataLoader(
    valid_dataset, batch_size=1, num_workers=0, persistent_workers=False
)

In [ ]:
model_cfg = dict(
    name="pointnet",
    n_cls=260,
    inp_dim=3,
    seg_hidden_dims=[512, 256],
    mlp2_hidden_dims=[64, 128, 1024],
    transf_hidden_dims1=[64, 128, 1024],
    transf_hidden_dims2=[512, 256],
)
trainer = TrainerSeg(
    train_loader,
    valid_loader,
    lr=0.001,
    device="cuda",
    weight_decay=0.0,
    num_epochs=300,
    save_dir=os.path.join(save_dir, "segmentation"),
    **model_cfg,
)
trainer.run()

Vizualize the results

In [ ]:
trainer.visualize()

In [ ]:
trainer.test()

Final task: save the notebook .ipynb file **with logs**, and create a zip with the checkpoints folder. You can send your results to geometricdeeplearning@protonmail.com